***Syntax natürlicher Sprachen, WS 2022/23***

---
# Übung 12 (Lösung)

In [1]:
import nltk
from nltk.corpus import treebank
from nltk.grammar import ProbabilisticProduction, PCFG
from collections import defaultdict

---
###  Herunterladen von Ressourcen

#### Laden Sie zunächst die Ressource „corpora/treebank“ mithilfe des NLTK-Download-Managers herunter, falls dieser noch nicht installiert ist.

In [2]:
# nltk.download()

---
## Aufgabe 1: Grammatikinduktion

#### In dieser Aufgabe soll vollautomatisch aus Daten (Syntaxbäumen) eine probabilistische, kontextfreie Grammatik erzeugt werden.


### Aufgabe 1.1: Von Daten zu Regelwahrscheinlichkeiten

#### Veranschaulichen Sie sich das Vorgehen zunächst an einem Beispiel. Gegeben sei folgende Grammatik:

In [3]:
cfg = """
S -> NP VP
VP -> V NP PP
VP -> V NP
NP -> DET N
NP -> NP PP
PP -> P NP

DET -> "the" | "a"
N -> "boy" | "woman" | "telescope"
V -> "saw"
P -> "with"
"""

Sie modelliert sehr einfache Sätze der Form SBJ saw OBJ mit optionaler Präpositionalphrase am Ende. Diese Präpositionalphrase kann entweder der näheren Bestimmung des Objekts oder der näheren Bestimmung der in der Verbalphrase ausgedrückten Handlung dienen.

**Für welche Regeln müssen wir die Wahrscheinlichkeiten berechnen, wenn wir mit statistischen Methoden untersuchen wollen, ob PPs häufiger Teil der VP oder Teil der NP sind?**

Approximieren Sie mittels vergleichbarer Konstruktionen in der Penn Treebank die Wahrscheinlichkeiten für die ersten beiden dieser Regeln.

In [4]:
counter = defaultdict(int)

for tree in treebank.parsed_sents():
    for prod in tree.productions():
        if prod.lhs() == nltk.grammar.Nonterminal('VP'):
            counter[prod] += 1
            
constructions = [ (k, counter[k]) for k in sorted(counter.keys(), key=counter.__getitem__, reverse=True) ]
constructions[:30]

[(VP -> TO VP, 1257),
 (VP -> VB NP, 805),
 (VP -> MD VP, 759),
 (VP -> VBD SBAR, 631),
 (VP -> VBZ VP, 459),
 (VP -> VBD NP, 378),
 (VP -> VBG NP, 375),
 (VP -> VBD VP, 361),
 (VP -> VBP VP, 337),
 (VP -> VBZ NP, 261),
 (VP -> VB VP, 258),
 (VP -> VBN NP, 250),
 (VP -> VP CC VP, 234),
 (VP -> VBD S, 223),
 (VP -> VBZ S, 215),
 (VP -> VBZ SBAR, 197),
 (VP -> VBP NP, 185),
 (VP -> VBN NP PP-CLR, 178),
 (VP -> VBN NP PP, 170),
 (VP -> VBZ NP-PRD, 163),
 (VP -> VB S, 155),
 (VP -> VBN S, 141),
 (VP -> VBP SBAR, 121),
 (VP -> VB PP-CLR, 107),
 (VP -> VBG S, 89),
 (VP -> VBP S, 88),
 (VP -> VB NP PP-CLR, 88),
 (VP -> VBZ ADJP-PRD, 87),
 (VP -> VBN VP, 84),
 (VP -> MD RB VP, 82)]

In [5]:
vp_with_pp = 178 + 170 + 88
vp_without_pp = 805 + 378 + 375 + 261 + 250 + 185

prob1 = vp_with_pp / (vp_with_pp + vp_without_pp)
prob2 = vp_without_pp / (vp_with_pp + vp_without_pp)

prob1, prob2

(0.1620817843866171, 0.8379182156133829)

---
### Aufgabe 1.2: Induktion von PCFG-Regeln aus der Penn-Treebank

#### Im Folgenden wollen wir vollautomatisch eine aus den Syntaxbäumen der Penn Treebank induzierte Grammatik erzeugen.

#### Füllen Sie die Lücken und versuchen Sie mithilfe Ihrer automatisch erstellten Grammatik die folgenden Sätze zu parsen:

In [6]:
test_sentences = [
    "the men saw a car .",
    "the woman gave the man a book .",
    "she gave a book to the man .",
    "yesterday , all my trouble seemed so far away ."
]

In [7]:
# Production count: the number of times a given production occurs
pcount = defaultdict(int)

# LHS-count: counts the number of times a given lhs occurs
lcount = defaultdict(int)

for tree in treebank.parsed_sents():
    for prod in tree.productions():
        pcount[prod] += 1
        lcount[prod.lhs()] += 1
        
productions = [
    ProbabilisticProduction(
        p.lhs(), p.rhs(),
        prob=pcount[p] / lcount[p.lhs()]
    )
    for p in pcount
]

start = nltk.Nonterminal('S')
grammar = PCFG(start, productions)
parser = nltk.ViterbiParser(grammar)

In [8]:
for s in test_sentences:
    for t in parser.parse(nltk.word_tokenize(s)):
        print(t.prob())
        t.pretty_print(unicodelines=True)

2.269940263066198e-15
                S                 
      ┌─────────┴───┬───────────┐  
      │             VP          │ 
      │         ┌───┴───┐       │  
    NP-SBJ      │       NP      │ 
 ┌────┴─────┐   │   ┌───┴───┐   │  
 DT        NNS VBD  DT      NN  . 
 │          │   │   │       │   │  
the        men saw  a      car  . 

8.44395440336237e-21
                       S                             
      ┌────────────────┴───────┬───────────────────┐  
      │                        VP                  │ 
      │           ┌────────┬───┴────────┐          │  
    NP-SBJ        │        NP         NP-TMP       │ 
 ┌────┴──────┐    │    ┌───┴───┐   ┌────┴─────┐    │  
 DT          NN  VBD   DT      NN  DT         NN   . 
 │           │    │    │       │   │          │    │  
the        woman gave the     man  a         book  . 

1.3157826588159793e-18
                     S                         
  ┌──────────────────┼───────────────────────┐  
  │                  VP   

---
## Aufgabe 2: Parent Annotation

#### *Parent Annotation* kann die Performanz einer CFG wesentlich verbessern. Schreiben Sie eine Funktion, die einen gegebenen Syntaxbaum dieser Optimierung unterzieht. Auf diese Art und Weise transformierte Bäume können dann wiederum zur Grammatikinduktion verwendet werden.

#### `parentHistory` soll dabei die Anzahl der Vorgänger sein, die zusätzlich zum direkten Elternknoten berücksichtigt werden. (Die Berücksichtigung dieses Parameters ist optional.)

#### `parentChar` soll ein Trennzeichen sein, das bei den neuen Knotenlabels zwischen dem ursprünglichen Knotenlabel und der Liste von Vorgängern eingefügt wird.

In [9]:
def parent_annotation(tree, parentHistory=0, parentChar="^"):
    def pa_rec(node, parents):
        originalNode = node.label() 
        parentString = (
            parentChar + '<' + '-'.join(parents) + '>'  
        )
        node.set_label(node.label() + parentString) 
        for child in node: 
            pa_rec(
                child,
                [originalNode] + parents[:parentHistory] 
            )
        return node
    return pa_rec(tree, []) 

#### Zur Überprüfung Ihrer Implementierung können Sie sie mit folgendem Beispielbaum testen:

In [10]:
test_tree = nltk.Tree(
    "S",
    [
        nltk.Tree("NP", [
            nltk.Tree("DET", []),
            nltk.Tree("N", [])
        ]),
        nltk.Tree("VP", [
            nltk.Tree("V", []),
            nltk.Tree("NP", [
                nltk.Tree("DET", []),
                nltk.Tree("N", [])
            ])
        ])
    ]
)

parent_annotation(test_tree).pretty_print(unicodelines=True)

                        S^<>                         
           ┌─────────────┴───────┐                    
           │                   VP^<S>                
           │             ┌───────┴────────┐           
         NP^<S>          │             NP^<VP>       
   ┌───────┴──────┐      │       ┌────────┴──────┐    
DET^<NP>        N^<NP> V^<VP> DET^<NP>         N^<NP>
   │              │      │       │               │    
  ...            ...    ...     ...             ...  

